<a href="https://colab.research.google.com/github/tsakailab/prml/blob/master/ipynb/ex_GaussianNaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ガウス単純ベイズ識別（Gaussian naive Bayes classification）

----

氏名：

学生番号：

----
基本課題（必須）

    1. 「★scikit-learn を用いたガウス単純ベイズ識別」まで実行すると，何を表す数値と図が出力されますか．
       また，表示される数値と図から何が言えますか．特に，Example 2について記述してください．

（ここに回答を書いてください）



    2. 「★事前確率の計算」と「★標本平均と標本分散の計算」を完成させてください．
       また，その実行結果が，「★scikit-learn を用いたガウス単純ベイズ識別」で表示される 
       "Prior probs"，"Mean"，"Variance" と一致することを確認してください．

（回答は「★事前確率の計算」と「★標本平均と標本分散の計算」に書いてください）



    3. 「★scikit-learn を用いたガウス単純ベイズ識別」または「★確率密度の比較」で表示される決定境界は，
       どのような曲線があり得るでしょうか．色々な設定でExampleを観察して確認してください．

（ここに回答を書いてください）  



    4.その他，気づいたこと，調べたことを書いてください．

（ここに回答を書いてください）



----

In [ ]:
#@title グラフを描くクラス `plot2cls` を定義します（理解不要）．
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import TwoSlopeNorm as tsn
import warnings
warnings.filterwarnings("ignore")#, category=np.VisibleDeprecationWarning)

class plot2cls:
    def __init__(self, X_train, y_train, decision_function=None, X_val=None, y_val=None, dx=0.02, cmap=plt.cm.bwr, xlim=None, ylim=None, margin=0.5, levels={0.0:'-'}, colors='k', bins=None):
        self.data = {'train': {'X': X_train, 'y': y_train}, 'val': {'X':X_val, 'y': y_val}}
        self.clf = decision_function
        if X_val is not None:
            xlim = [X_val[:, 0].min(), X_val[:, 0].max()]
            ylim = [X_val[:, 1].min(), X_val[:, 1].max()]
        else:
            xlim = [float('inf'), - float('inf')]
            ylim = [float('inf'), - float('inf')]
        xlim[0] = min(xlim[0], X_train[:, 0].min()) - margin
        xlim[1] = max(xlim[1], X_train[:, 0].max()) + margin
        ylim[0] = min(ylim[0], X_train[:, 1].min()) - margin
        ylim[1] = max(ylim[1], X_train[:, 1].max()) + margin
        if bins is None:
            bins = len(y_train) // 4
        self.layout = {'dx': dx, 'cmap': cmap, 'xlim': xlim, 'ylim': ylim, 'levels': levels, 'colors': colors, 'bins': bins}


    def _data_layout(self, ax, bbox_to_anchor, loc):
        ax.set_xlim(self.layout['xlim'][0], self.layout['xlim'][1])
        ax.set_ylim(self.layout['ylim'][0], self.layout['ylim'][1])
        #plt.axis('tight')
        ax.set_xlabel('x1', fontsize=16)
        ax.set_ylabel('x2', fontsize=16)
        ax.tick_params(axis='x', labelsize=16)
        ax.tick_params(axis='y', labelsize=16)
        ax.set_aspect('equal')
        #ax.legend(bbox_to_anchor=(1.5,1.0), loc="upper right", fontsize=16, frameon=True)
        ax.legend(bbox_to_anchor=bbox_to_anchor, loc=loc, fontsize=16, frameon=True)
        if self.data['val']['X'] is not None and self.data['val']['y'] is not None:
            ax.legend(bbox_to_anchor=bbox_to_anchor, loc=loc, fontsize=16, frameon=True, ncol=2)
            ax.get_legend().legendHandles[2].set_color('k')
            ax.get_legend().legendHandles[3].set_color('k')
        plt.tight_layout()


    def _scatter(self, ax, X, y, c=['r', 'b'], marker=['s', 'o'], label=None, alpha=1):
        ax.scatter(X[y>0, 0], X[y>0, 1], c=c[0],  marker=marker[0], cmap=self.layout['cmap'], edgecolors='k', label=label+'(+)', alpha=alpha)
        ax.scatter(X[y<=0, 0], X[y<=0, 1], c=c[1], marker=marker[1], cmap=self.layout['cmap'], edgecolors='k', label=label+'(-)', alpha=alpha)


    def _put_data(self, ax, alpha):
        if self.clf is not None and self.layout['levels'] is not None:
            xx, yy = np.meshgrid(np.arange(self.layout['xlim'][0], self.layout['xlim'][1], self.layout['dx']), np.arange(self.layout['ylim'][0], self.layout['ylim'][1], self.layout['dx']))

            # Show prediction by color by assigning a color to each point in the mesh [x_min, x_max]x[y_min, y_max].
            Z = self.clf(np.c_[xx.ravel(), yy.ravel()])
            # Put the result into a color plot
            Z = Z.reshape(xx.shape)
            norm = tsn(vmin=np.minimum(Z[:].min(),-1e-6), vcenter=0, vmax=np.maximum(Z[:].max(),1e-6))
            if self.layout['cmap'] is not None:
                ax.pcolor(xx, yy, Z, cmap=self.layout['cmap'], alpha=0.1, edgecolors=None, norm=norm)
            lvls = list(self.layout['levels'].keys())
            linestyles = list(self.layout['levels'].values())
            ax.contour(xx, yy, Z, levels=lvls, colors=self.layout['colors'], linestyles=linestyles, alpha=0.5)

        self._scatter(ax, self.data['train']['X'], self.data['train']['y'], label='train', alpha=alpha)
        if self.data['val']['X'] is not None and self.data['val']['y'] is not None:
            self._scatter(ax, self.data['val']['X'], self.data['val']['y'], c=['k', 'k'], label='val', alpha=alpha*0.2)


    def plot_data(self, bbox_to_anchor=(1,0), loc="lower left"):
        ax = plt.figure(figsize=(8,8))
        ax = plt.axes()
        self._put_data(ax, alpha=1)
        self._data_layout(ax, bbox_to_anchor=bbox_to_anchor, loc=loc)


    def _hist_layout(self, ax, bbox_to_anchor, loc, xlabel):
        ax.set_xlabel(xlabel, fontsize=16)
        ax.set_ylabel("Frequency", fontsize=16)
        #ax.axis('tight')
        ax.tick_params(axis='x', labelsize=16)
        ax.tick_params(axis='y', labelsize=16)
        #plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
        from matplotlib.ticker import FormatStrFormatter
        ax.yaxis.set_major_formatter(FormatStrFormatter('%1.0f'))
        #ax.set_aspect(1)
        ax.legend(bbox_to_anchor=bbox_to_anchor, loc=loc, fontsize=16, frameon=True)
        if self.data['val']['X'] is not None and self.data['val']['y'] is not None:
            ax.legend(bbox_to_anchor=bbox_to_anchor, loc=loc, fontsize=16, frameon=True, ncol=2)
            #ax.legend(ncol=2)
        plt.tight_layout()


    def _make_hist(self, ax):
        if self.clf is None:
            return
        pred = self.clf(self.data['train']['X'])
        gt = self.data['train']['y']
        ax.hist( [ pred[gt>0], pred[gt<=0] ], bins=self.layout['bins'], histtype='stepfilled', density=False, alpha=0.5, color=['r', 'b'], label=['train(+)', 'train(-)'])
        if self.data['val']['X'] is not None and self.data['val']['y'] is not None:
            pred = self.clf(self.data['val']['X'])
            gt = self.data['val']['y']
            ax.hist( [ pred[gt>0], pred[gt<=0] ], bins=self.layout['bins'], histtype='stepfilled', density=False, alpha=0.3, color=['r', 'b'], label=['val(+)', 'val(-)'])


    def plot_hist(self, loc="lower left", bbox_to_anchor=(0,1), xlabel="$g(x)$"):
        ax = plt.figure(figsize=(6,6))
        ax = plt.axes()
        self._make_hist(ax)
        self._hist_layout(ax,bbox_to_anchor, loc, xlabel)


    def plot_clf(self, loc="lower left", bbox_to_anchor=(0,1), xlabel="$g(x)$"):
        if self.clf is None:
            return

        fig, axes = plt.subplots(figsize=(12,6), nrows=1, ncols=2)
        #fig, axes = plt.subplots(nrows=1, ncols=2)
        ax = axes[0]
        self._put_data(ax, alpha=1)
        self._data_layout(ax, bbox_to_anchor, loc)

        #fig.set_figwidth(12)
        #fig.set_figheight(8)
        ax = axes[1]
        self._make_hist(ax)
        self._hist_layout(ax, bbox_to_anchor, loc, xlabel)


### 実験用のデータ（2次元，2クラス）を生成します．
- Example 2～5 からひとつ選んで実行してください．

In [ ]:
# Example 2: draw npos and nneg points from the Gaussian distribution for each class
npos = 100
nneg = 500
np.random.seed(321)
X_all = np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]
# [1,1,...,1,-1,-1,...,-1]
y_all = np.array([1] * npos + [-1] * nneg)

In [ ]:
# Example 3: create moons using sklearn
from sklearn.datasets import make_moons
X_all, y_all = make_moons(n_samples=300, noise=0.3, random_state=0)
y_all[y_all==0] = -1

In [ ]:
# Example 4: create circles using sklearn
from sklearn.datasets import make_circles
X_all, y_all = make_circles(n_samples=300, noise=0.2, random_state=0, factor=0.3)
y_all[y_all==0] = -1

In [ ]:
# Example 5: combined dataset
from sklearn.datasets import make_moons
X_all, y_all = make_moons(n_samples=100, noise=0.2, random_state=0)
y_all[y_all==0] = -1
npos, nneg = 30, 300
np.random.seed(321)
X_all = np.vstack((2*X_all+np.array([4,-2]), np.r_[np.random.randn(npos, 2) + [3, 3], np.random.randn(nneg, 2)]))
# [1,1,...,1,-1,-1,...,-1]
y_all = np.hstack((y_all, np.array([1] * npos + [-1] * nneg)))

### 訓練データ集合 `(X,y)` と検証データ集合 `(X_val,y_val)` に分けます．

In [ ]:
from sklearn.model_selection import train_test_split

X, X_val, y, y_val = train_test_split(X_all, y_all, test_size=0.2, random_state=0)
print("#train&val, #test =", X.shape[0], ",", X_val.shape[0])

In [ ]:
#@title データを表示します．
print("#X_all =", X_all.shape[0])
plot2cls(X, y, X_val=X_val, y_val=y_val).plot_data()

### ★scikit-learn を用いたガウス単純ベイズ識別

[sklearn.naive_bayes.GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)を使用した結果を出力します．

In [ ]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X,y)

# クラス数（number of classes）
print("# of classes: ", len(model.class_count_))

# 各クラスの事前確率（prior probability of each class）
print("Prior probs: ", model.class_prior_)

# 各クラスの各特徴の標本平均と標本分散（mean and variance of each feature per class）
print("Mean: ", model.theta_.tolist())
print("Variance: ", model.var_.tolist())

print("Accuracy on training data: ", model.score(X, y))
print("Accuracy on validation data: ", model.score(X_val, y_val))

dec = lambda X: np.diff(model.predict_proba(X))
plot2cls(X, y, dec, X_val, y_val).plot_clf()

In [ ]:
#@title 混同行列（行：正解，列：予測）
from sklearn import metrics
cm = metrics.confusion_matrix(y_val, model.predict(X_val))
print(cm)

from sklearn.metrics import classification_report
print(classification_report(y_val, model.predict(X_val)))

## ガウス単純ベイズ識別の計算を確認しましょう．

生成確率密度 $P(x|Y)=P(x_1|Y)P(x_2|Y)$, 

各クラスの各特徴の生成確率密度 $P(x_i|Y) = \frac{1}{\sqrt{2\pi}\sigma_{i|Y}}\exp\left\{-\frac{(x_i-\mu_{i|Y})^2}{2\sigma_{i|Y}^2}\right\}$ （ただし，$Y\in\{+,-\}$, $i\in\{1,2\}$）

- **★事前確率の計算**：各クラスのデータ数の割合P(Y)を求めます．
- **★標本平均と標本分散の計算**：特徴が独立に正規分布することを仮定して，各クラスの訓練データから各特徴の標本平均（sample mean）$\mu_{1|+}$, $\mu_{2|+}$, $\mu_{1|-}$, $\mu_{2|-}$と，標本分散（sample variance）$\sigma_{1|+}$, $\sigma_{2|+}$, $\sigma_{1|-}$, $\sigma_{2|-}$を求めます（統計的学習）．
- **★確率密度の比較**：各クラスの事後確率密度を比較します（統計的推論）．事後確率密度の代わりに同時確率密度 $P(x,Y) = P(x|Y)P(Y)$ を計算します．

### ★事前確率の計算

- 2クラスのデータは `X_p = X[y>0]` と `X_n = X[y<=0]` です．その各行は2次元の特徴ベクトル x です．
- クラス$+$ (`y>0`) と$-$ (`y<=0`) のデータ数の割合を `Prior[0]` と`Prior[1]` とします．

In [ ]:
X_p = X[y>0]
X_n = X[y<=0]
print(X_p.shape, X_n.shape)

Prior = np.zeros(2)
Prior[0], Prior[1] = ''' 事前確率を計算してください '''

print("Prior =", Prior)

### ★標本平均と標本分散の計算

- 2クラスのデータは `X_p = X[y>0]` と `X_n = X[y<=0]` です．その各行は2次元の特徴ベクトルです．
- 標本平均ベクトルをそれぞれ `mu_p`，`mu_n` とします（`mu_p.shape` と `mu_n.shape` は `(2,)` です）．それぞれ，標本平均ベクトル$[\mu_{1|+},\mu_{2|+}]^\top$と$[\mu_{1|-},\mu_{2|-}]^\top$を表します．
- `X_p` と `X_n` の各特徴の標本分散をそれぞれ `s_p`，`s_n` とします．`s_p.shape` と `s_n.shape` は `(2,)` です．

In [ ]:
mu_p = ''' X_p から標本平均を計算してください ''' 
print("[mu1_p, mu2_p] = ", mu_p)

s_p = ''' X_p から標本分散を計算してください '''
print("[s11_p, s22_p] = ", s_p)

mu_n = ''' X_p から標本平均を計算してください '''
print("[mu1_n, mu2_n] = ", mu_n)

s_n = ''' X_n から標本分散を計算してください '''
print("[s11_n, s22_n] = ", s_n)


### ★確率密度の比較

- 計算した事前確率，標本平均，標本分散を用いて，データと共に正規分布と境界を図示します．
- 表示した領域内の多数の点で，各クラスの生成分布の確率密度を計算し（統計的推論），確率密度を色の濃さと等高線で表します．

In [ ]:
#@title 各クラスの2次元ガウス分布を図示する関数 `plot2d_GaussianNB` を定義します（理解不要）．
import numpy as np
# Independent bivariate normal joint density
def N2d(x, mu, variance):
    return np.exp(-0.5 * (x[:,0] - mu[0])**2 / variance[0])  *  np.exp(-0.5 * (x[:,1] - mu[1])**2 / variance[1])  /  (2. * np.pi * np.sqrt(variance[0]*variance[1]))
    #return np.exp(-0.5 * (x[:,0] - mu[0])**2 / variance[0])  *  np.exp(-0.5 * (x[:,1] - mu[1])**2 / variance[1])

from matplotlib import pyplot as plt
# Visualization of the estimated distributions
def plot2d_GaussianNB(X_train, y_train, pri, mus, vars, X_val=None, y_val=None, cmap=None, xlim=None, ylim=None, levels=None, linestyles=None, markers=None, colors=None):

    plt.figure(figsize=(8,8))
    ax = plt.axes()

    dlim = np.sqrt(vars).sum(axis=0)/2
    dxlim, dylim = dlim[0], dlim[1]

    if xlim is None:
        xlim = [X_train[:, 0].min() - dxlim, X_train[:, 0].max() + dxlim]
    if ylim is None:
        ylim = [X_train[:, 1].min() - dylim, X_train[:, 1].max() + dylim]

    xg, yg = np.arange(xlim[0], xlim[1], (xlim[1]-xlim[0])/300.), np.arange(ylim[0], ylim[1], (ylim[1]-ylim[0])/300.)
    xx, yy = np.meshgrid(xg, yg)

    if cmap is None:
        cmap = ['Reds', 'Blues', 'Greens', 'BuPu', 'RdPu', 'YlGn']
    if markers is None:
        markers = ['o', 's', '^', '*', '+', 'x']
    if colors is None:
        colors = ['b', 'r', 'g', 'c', 'm', 'y']
    ncmap, nm, nc = len(cmap), len(markers), len(colors)

    ncls = len(pri)
    vmax = 1.  / (2. * np.pi * np.sqrt(vars[:,0] * vars[:,1]))
    for k in range(ncls):
        #mean and variance of each feature per class
        mu = mus[k,:]
        variance = vars[k,:]
        Z = N2d(np.c_[xx.ravel(),yy.ravel()], mu, variance)

        # Put the result into a color plot
        #ax.pcolor(xx, yy, Z, cmap=cmap[k], alpha=0.5, edgecolors=None)
        Zm = np.ma.masked_array(Z, Z < 0.03*vmax[k])
        ax.pcolorfast(xg, yg, Zm.reshape(xx.shape), cmap=cmap[k%ncmap], alpha=0.5)
        if levels is not None:
            ax.contour(xx, yy, Z.reshape(xx.shape), levels=levels, colors='k', linestyles=linestyles, alpha=0.2)
        else:
            levels = np.arange(0, vmax[k], vmax[k]/8.)
            ax.contour(xx, yy, Z.reshape(xx.shape), levels=levels, colors='k', linestyles=linestyles, alpha=0.2)

    if ncls == 2:
        decision_function = lambda x: N2d(x, mus[1], vars[1])*pri[1] - N2d(x, mus[0], vars[0])*pri[0]
        Z = decision_function(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        ax.contour(xx, yy, Z.reshape(xx.shape), levels=[0.], colors='k', linestyles=['--'], alpha=1)

    # Plot also the training points
    y = np.unique(y_train)
    for k in range(ncls):
        ax.scatter(X_train[y_train==y[k], 0], X_train[y_train==y[k], 1], c=colors[k%nc],  marker=markers[k%nm], cmap=cmap[k%ncmap], edgecolors='k', label='Training data', alpha=1)
    # and val points if given
    if X_val is not None and y_val is not None:
        y = np.unique(y_train)
        for k in range(ncls):
            ax.scatter(X_val[y_val==y[k], 0], X_val[y_val==y[k], 1], c='k', marker=markers[k%nm], cmap=cmap[k], edgecolors='k', label='Val data', alpha=0.2)
        plt.legend(loc="upper right", fontsize=16, frameon=True, bbox_to_anchor=(0,1))

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    plt.axis('tight')
    plt.xlabel('x1', fontsize=16)
    plt.ylabel('x2', fontsize=16)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.gca().set_aspect('equal')
    plt.tight_layout()
    #plt.savefig('rnd2d_ex1_GNB.png', transparent=True,dpi=300)

In [ ]:
plot2d_GaussianNB(X, y, Prior, np.c_[mu_p, mu_n].T, np.c_[s_p, s_n].T, X_val, y_val)

----
## 付録：2変量正規分布（Bivariate normal distribution）

${\rm N}(\mathbf{x};\mathbf{\mu},\mathbf{\Sigma})=\dfrac{1}{\sqrt{(2\pi)^m|\mathbf{\Sigma}|}}\,e^{-\frac{1}{2}(\mathbf{x}-\mathbf{\mu})^\top\Sigma^{-1}(\mathbf{x}-\mathbf{\mu})}$

$\mathbf{x}=\left[\matrix{x_1\\x_2}\right]$, $\mathbf{\mu}=\left[\matrix{\mu_1\\\mu_2}\right]$, $\mathbf{\Sigma}=\left[\matrix{s_{11}&s_{12}\\s_{21}&s_{22}}\right]$ 

In [ ]:
#@title  { run: "auto" }
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

mu1 = 0 #@param {type:"slider", min:-5, max:5, step:0.1}
mu2 = 0 #@param {type:"slider", min:-5, max:5, step:0.1}
s11 = 1 #@param {type:"slider", min:0.00, max:4, step:0.1}
s12 = 0 #@param {type:"slider", min:-4, max:4, step:0.1}
s22 = 1 #@param {type:"slider", min:0.00, max:4, step:0.1}

if s12*s12 >= s11*s22:
    print("s12*s12 must be smaller than s11*s22")
    s12 = np.sign(s12)*(s11*s22-1e-8)

xmin, xmax = -5,5
x1, x2 = np.mgrid[xmin:xmax:.01, xmin:xmax:.01]
x = np.dstack((x1, x2))

prob = stats.multivariate_normal([mu1, mu2], [[s11, s12], [s12, s22]])

plt.figure()
ax = plt.axes()

ax.contourf(x1, x2, prob.pdf(x), alpha=0.9, cmap=plt.cm.Greens)
label = '$\\mu=$['+"{:.2f}".format(mu1)+', '+"{:.2f}".format(mu2)+']\n$\\Sigma=$[['+"{:.2f}".format(s11)+', '+"{:.2f}".format(s12)+']\n        ['+"{:.2f}".format(s12)+', '+"{:.2f}".format(s22)+']]'
plt.text(1, 3, label, fontsize=16)
plt.xlim(xmin, xmax)
plt.ylim(xmin, xmax)
plt.grid(True, ls='--', lw=0.5, color='k')
plt.xlabel("$x_1$", fontsize=20)
plt.ylabel("$x_2$", fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
ax.set_aspect('equal')
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
plt.tight_layout()

お疲れさまでした．

--------